# **Projeto: Generative AI - Avaliação de Modelo | Benchmark com Stanford Question Answering Dataset (SQuAD)**

**Autor**: Pedro Henrique Pedroso da Cruz

Este *notebook* contém exemplos de código para casos de validação de Modelos Gen AI através da abordagem de benchmarck, comparando com um dataset já preparado.

*O código é um exemplo simples e direto para demonstrar uma das estratégias de avaliação de modelo, e não deve ser utilizado em ambiente de produção. Para boas práticas toda e qualquer Chave Key não utilizar estar Hard Code.*

# Código

Configuração Ambiente

Vamos escolher um modelo Hugging Face chamado "distilbert-base-uncased-distilled-squad".

Mais informações acessar o link:

https://huggingface.co/distilbert/distilbert-base-uncased-distilled-squad


In [1]:
# Instalar as bibliotecas necessárias
!pip install transformers datasets

# Importar as bibliotecas necessárias
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import numpy as np
from sklearn.metrics import f1_score, accuracy_score


# Carregar o modelo e o tokenizer da Hugging Face
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Configurar o pipeline de QA
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Carregando Dataset

In [2]:
# Carregar o dataset SQuAD
squad_dataset = load_dataset("squad")

Apresentar o Dataset

In [3]:
# Função para exibir algumas linhas do dataset
def display_samples(num_samples=10):
    samples = squad_dataset['validation'].select(range(num_samples))
    for idx, sample in enumerate(samples):
        print(f"Index: {idx}")
        print(f"Question: {sample['question']}")
        print(f"Context: {sample['context'][:200]}...")  # Limitar o contexto para melhor visualização
        print(f"Answer: {sample['answers']['text'][0]}\n")

# Exibir 20 amostras do dataset
display_samples(20)

Index: 0
Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated...
Answer: Denver Broncos

Index: 1
Question: Which NFL team represented the NFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated...
Answer: Carolina Panthers

Index: 2
Question: Where did Super Bowl 50 take place?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated...
Answer: Santa Clara, California

Index: 3
Question: Which NFL team won Super Bowl 50?
Context: Super Bowl 50 wa

Função para avaliação do modelo que compara as respostas geradas com as respostas corretas do SQuAD.

In [4]:
# Função para avaliar uma amostra
def evaluate_sample(sample):
    context = sample['context']
    question = sample['question']
    answer = sample['answers']['text'][0]

    prediction = qa_pipeline({'context': context, 'question': question})['answer']

    return question, prediction, answer

Método para calcular as métricas

In [5]:
# Função para calcular métricas de avaliação
def compute_metrics(predictions, references):
    f1_scores = []
    accuracies = []

    for pred, ref in zip(predictions, references):
        pred_tokens = set(pred.split())
        ref_tokens = set(ref.split())

        # Calcular F1 Score
        common_tokens = pred_tokens & ref_tokens
        if len(common_tokens) == 0:
            f1_scores.append(0)
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(ref_tokens)
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_scores.append(f1)

        # Calcular Exatidão
        accuracies.append(int(pred == ref))

    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)

    return avg_f1, avg_accuracy

# Análise do resultados
Validando um número especifico de questões

In [6]:
# Função para validar um número específico de questões
def validate_questions(num_questions=10):
    samples = squad_dataset['validation'].select(range(num_questions))
    predictions = []
    references = []
    questions = []

    for sample in samples:
        question, pred, ref = evaluate_sample(sample)
        questions.append(question)
        predictions.append(pred)
        references.append(ref)
        print(f"Question: {question}")
        print(f"Generated Answer: {pred}")
        print(f"Correct Answer: {ref}\n")

    # Calcular e exibir as métricas
    avg_f1, avg_accuracy = compute_metrics(predictions, references)
    print(f"F1 Score: {avg_f1:.2f}")
    print(f"Accuracy: {avg_accuracy:.2f}")

# Validar um número específico de questões
validate_questions(num_questions=10)

Question: Which NFL team represented the AFC at Super Bowl 50?
Generated Answer: Denver Broncos
Correct Answer: Denver Broncos

Question: Which NFL team represented the NFC at Super Bowl 50?
Generated Answer: Carolina Panthers
Correct Answer: Carolina Panthers

Question: Where did Super Bowl 50 take place?
Generated Answer: Levi's Stadium
Correct Answer: Santa Clara, California

Question: Which NFL team won Super Bowl 50?
Generated Answer: Denver Broncos
Correct Answer: Denver Broncos

Question: What color was used to emphasize the 50th anniversary of the Super Bowl?
Generated Answer: gold
Correct Answer: gold

Question: What was the theme of Super Bowl 50?
Generated Answer: Arabic numerals
Correct Answer: "golden anniversary"

Question: What day was the game played on?
Generated Answer: February 7, 2016
Correct Answer: February 7, 2016

Question: What is the AFC short for?
Generated Answer: American Football Conference
Correct Answer: American Football Conference

Question: What was t

# Explicação das métricas

**F1 Score**

O F1 Score é uma métrica de avaliação que considera tanto a precisão (precision) quanto a revocação (recall) de um modelo. Ele é aplicável para modelos de QA, e outros casos.


*Precisão (Precision)*: A proporção de respostas corretas entre as respostas fornecidas pelo modelo.

*Revocação (Recall)*: A proporção de respostas corretas identificadas pelo modelo entre todas as respostas corretas possíveis.
Um **F1 Score de 0.70** indica que o modelo tem um equilíbrio razoável entre precisão e revocação. No contexto de QA, isso significa que,** em média, 70% das palavras ou frases geradas pelo modelo como respostas são corretas e relevantes**.

**Accuracy**

A accuracy mede a proporção de respostas totalmente corretas em relação ao total de perguntas.

Uma accuracy de **0.70 **significa que **70% das respostas fornecidas pelo modelo são exatamente iguais às respostas corretas do dataset SQuAD**.

# Interpretação do Resultado

**F1 Score de 0.70**: Esse valor indica que o modelo é bastante competente em fornecer respostas que contêm os elementos corretos. No entanto, há espaço para melhoria, pois 30% das respostas podem estar faltando informações importantes ou incluindo informações irrelevantes.
Vejamos um exemplo de resposta diferente:

Question: What was the theme of Super Bowl 50?

Generated Answer: Arabic numerals

Correct Answer: "golden anniversary" *texto em itálico*

**Accuracy de 0.70**: Este valor indica que o modelo fornece exatamente a resposta correta em 70% das perguntas. Isso é um bom resultado, especialmente considerando a dificuldade do SQuAD, que exige precisão e exatidão nas respostas.

Aqui estamos validando apenas 10 questões, para o ideal seria validar o conjunto completo, existindo ainda a possibilidade de adicionar suas questões e respostas para o case de negócio.